In [1]:
import math
import collections
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import utils

In [2]:
MonthlyStatus = collections.namedtuple('MonthlyStatus',
                                       ['mortgage_amount', 'price_index', 'price_real_estate'])


def get_price_index(year_growth_rate, month):
    return math.pow(1 + year_growth_rate/12, month)


class StockPurchase:
    def __init__(self, amount, purchase_price):
        self.amount = amount
        self.purchase_price = purchase_price
        self.current_price = purchase_price
        self.units = self.amount / self.purchase_price
        self.value = self.units * self.current_price
    
    def update_value(self, current_price):
        self.current_price = current_price
        self.value = self.units * self.current_price
        return self
    
    def to_dict(self):
        return {'amount': self.amount,
                'purchase_price': self.purchase_price,
                'value': self.value}


class Portfolio:
    def __init__(self):
        self.purchases = []
    
    def purchase(self, amount, purchase_price):
        self.purchases.append(StockPurchase(amount, purchase_price))
        return self
    
    def update_values(self, current_price):
        for purchase in self.purchases:
            purchase.update_value(current_price)
        return self


class Simulation:
    def __init__(self, scenario, purchase_price, mortgage, portfolio):
        self.scenario = scenario
        self.purchase_price = purchase_price
        self.mortgage = mortgage
        self.portfolio = portfolio
        self.mortgage_history = []
        self.payment_history = []
    
    def get_monthly_status(self, month):
        mortgage_amount = self.mortgage.mortgage_amount
        price_index = get_price_index(self.scenario['growth_rate_real_estate'], month)
        current_price = (self.purchase_price + (self.purchase_price * (price_index - 1)))
        return MonthlyStatus(mortgage_amount, price_index, current_price)

    def run(self):
        i = 0
        max_periods = self.mortgage.maturity * self.mortgage.n_periods
        new_monthly_payment_amount = self.mortgage.monthly_payment + self.scenario['mortgage_overpay_month']
        self.mortgage.update_monthly_payment_amount(new_monthly_payment_amount)
        while self.mortgage.mortgage_amount > 0 and i <= max_periods:
            self.mortgage_history.append(self.get_monthly_status(i))
            self.payment_history.append(self.mortgage.get_next_payment())
            current_stock_price = (get_price_index(self.scenario['growth_rate_stocks'], i)
                                   * self.scenario['stock_price'])
            self.portfolio.purchase(self.scenario['mortgage_overpay_month'], current_stock_price)
            self.portfolio.update_values(current_stock_price)
            i += 1

In [3]:
portf = Portfolio()
init_price = 100
year_growth_rate = 0.05
for i in range(12):
    curr_price = get_price_index(year_growth_rate, i) * init_price
    portf.purchase(5000, curr_price)
    portf.update_values(curr_price)

In [4]:
pd.DataFrame.from_records([el.to_dict() for el in portf.purchases])

,amount,purchase_price,value
0,5000,100.000000,5234.001151
1,5000,100.416667,5212.283304
2,5000,100.835069,5190.655573
3,5000,101.255216,5169.117583
4,5000,101.677112,5147.668962
5,5000,102.100767,5126.309340
6,5000,102.526187,5105.038347
7,5000,102.953379,5083.855615
8,5000,103.382352,5062.760778
9,5000,103.813111,5041.753472


In [5]:
scenario_1 = {'growth_rate_real_estate': 0.05,
              'growth_rate_stocks': 0.05,
              'stock_price': 100,
              'currency_exchange_rate': 8.5,
              'mortgage_overpay_month': 5000}

port = Portfolio()
mort = utils.Mortgage(50, 0.0305, 3.6e6, 30, 12)
sim = Simulation(scenario_1, 4.2e6, mort, port)

In [6]:
sim.run()

In [7]:
result = (pd.DataFrame(sim.payment_history)
          .join(pd.DataFrame(sim.mortgage_history))
          .join(pd.DataFrame.from_records([el.to_dict() for el in sim.portfolio.purchases])))

In [15]:
result['monthly_payment'] = result.interest_amount + result.capital_downpayment_amount + result.fee
result['stock_profit'] = result.value - result.purchase_price

In [16]:
result.head()

,fee,interest_amount,capital_downpayment_amount,mortgage_amount,price_index,price_real_estate,amount,purchase_price,value,monthly_payment,stock_profit
0,50,9150.000000,11124.995854,3.600000e+06,1.000000,4.200000e+06,5000,100.000000,13339.483312,20324.995854,13239.483312
1,50,9121.723969,11153.271885,3.588875e+06,1.004167,4.217500e+06,5000,100.416667,13284.132759,20324.995854,13183.716092
2,50,9093.376069,11181.619785,3.577722e+06,1.008351,4.235073e+06,5000,100.835069,13229.011876,20324.995854,13128.176807
3,50,9064.956119,11210.039735,3.566540e+06,1.012552,4.252719e+06,5000,101.255216,13174.119711,20324.995854,13072.864495
4,50,9036.463935,11238.531919,3.555330e+06,1.016771,4.270439e+06,5000,101.677112,13119.455314,20324.995854,13017.778201


In [10]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 10 columns):
fee                           237 non-null int64
interest_amount               237 non-null float64
capital_downpayment_amount    237 non-null float64
mortgage_amount               237 non-null float64
price_index                   237 non-null float64
price_real_estate             237 non-null float64
amount                        237 non-null int64
purchase_price                237 non-null float64
value                         237 non-null float64
monthly_payment               237 non-null float64
dtypes: float64(8), int64(2)
memory usage: 18.6 KB


In [11]:
point_in_time = 12
paid_interests = result.interest_amount.iloc[:point_in_time].sum() + result.fee.iloc[:point_in_time].sum()
profit = result.price_real_estate.iloc[point_in_time] - result.price_real_estate.iloc[0]
equity = result.capital_downpayment_amount.iloc[:point_in_time].sum()
deposit = result.price_real_estate.iloc[0] - result.mortgage_amount[0]
print('Paid interests to date: {:.0f}'.format(paid_interests))
print('Profit in case of sale: {:.0f}'.format(profit))
print('Equity to date: {:.0f}'.format(equity))

Paid interests to date: 108518
Profit in case of sale: 214880
Equity to date: 135382


In [12]:
total_revenue = equity + profit + deposit
print('Total revenue from mortgage: {:.0f}'.format(total_revenue))

Total revenue: 950262


In [17]:
print('Total revenue from stocks: {:.0f}'.format(result.amount.sum() + result.stock_profit.sum()))
print('Total profit from stocks: {:.0f}'.format(result.stock_profit.sum()))

Total revenue from stocks: 3159519
Total profit from stocks: 1974519
